## Catering Event Planner

In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent, SocietyOfMindAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')



## Inner Team

In [2]:
# Inner Agent Team

planner_agent = AssistantAgent(
    name = 'PlannerAgent',
    model_client=model_client,
    description='An agent that interprests the user request and build a structured food menu',
    system_message="""
    You are a meticulous PlannerAgent specializing in event catering. 
    Your primary function is to interpret a user's high-level event brief, including the number of guests, dietary restrictions, event type (e.g., formal, casual), and food budget. 
    Break this information down into a structured, executable plan. 
    Your output should be a clear, actionable menu outline (e.g., "one appetizer, one main course, one dessert"). 
    Pass this plan to the 'RecipeAgent' to begin the creative process. 
    Do not create recipes yourself; your job is to set the stage.
    """
)

recipe_agent = AssistantAgent(
    name = 'RecipeAgent',
    model_client=model_client,
    description="An agent that create receipe based on the food menu",
    system_message="""
    You are a creative and knowledgeable RecipeAgent. 
    Your role is to generate detailed, appealing recipes based on the menu outline provided by the PlannerAgent. 
    Each recipe must include a list of ingredients, precise measurements, and step-by-step instructions. 
    You must adhere strictly to the dietary restrictions and event style. 
    After creating the recipes, pass the full menu to the CritiqueAgent for a quality check.
    """
)

critique_agent = AssistantAgent(
    name = 'CritiqueAgent',
    model_client=model_client,
    description = 'An agent that evaluate the menu and receipes',
    system_message="""
    You are the CritiqueAgent, a practical and experienced culinary expert. 
    Your job is to rigorously evaluate the menu and recipes from the RecipeAgent. 
    
    Check for potential issues:

    1. Feasibility: Is the menu too complex for the given time and resources?

    2. Ingredient Conflicts: Do multiple dishes require the same piece of equipment (e.g., the oven) at the same time?

    3. Budget Adherence: Does the menu align with the per-person food budget?

    4. Consistency: Does the menu flow well? Are there any odd flavor combinations?
    
    Your final output should be either an approval of the menu or a detailed list of suggestions for the RecipeAgent to correct. 
    
    Once the menu is approved, you will signal the UserProxyAgent to present the menu to the human for review.
    """
)

user_agent = UserProxyAgent(
    name = 'user',
    description='A proxy agent that represent the user',
    input_func=input
)

In [3]:
## Termination Condition
inner_team_termination = TextMentionTermination('APPROVE') | MaxMessageTermination(13)

culinary_team = RoundRobinGroupChat(
    participants=[planner_agent,recipe_agent,critique_agent, user_agent],
    termination_condition=inner_team_termination
)


In [6]:
from autogen_agentchat.ui import Console
task = '''
Please plan a menu for a formal dinner party for 30 people.
The guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors'''

await Console(culinary_team.run_stream(task = task))

---------- TextMessage (user) ----------

Please plan a menu for a formal dinner party for 30 people.
The guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors
---------- TextMessage (PlannerAgent) ----------
**Event Overview:**
- **Event Type:** Formal Dinner Party
- **Number of Guests:** 30
- **Dietary Restrictions:** None
- **Menu Preferences:** A mix of textures and flavors

**Menu Outline:**

1. **Appetizer Course:**
   - *Option 1:* Chilled Cucumber Avocado Soup with a touch of lime, served with a textural garnish of croutons.
   - *Option 2:* Stuffed Mushrooms with a creamy ricotta mixture topped with crispy breadcrumbs.

2. **Salad Course:**
   - *Option 1:* Mixed Greens Salad with candied walnuts, goat cheese, and a balsamic vinaigrette for a combination of crunch and creaminess. 
   - *Option 2:* Roasted Beet Salad with feta cheese and citrus segments for balance of earthiness and sweetness.

3. **Main Course:**
   - *Option 1:* Her

TaskResult(messages=[TextMessage(id='5917762b-2cfa-48b5-a833-602168767f01', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 7, 25, 59, 475435, tzinfo=datetime.timezone.utc), content='\nPlease plan a menu for a formal dinner party for 30 people.\nThe guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors', type='TextMessage'), TextMessage(id='09f1764a-0981-4ece-a95c-4b9debdb681a', source='PlannerAgent', models_usage=RequestUsage(prompt_tokens=170, completion_tokens=396), metadata={}, created_at=datetime.datetime(2025, 8, 8, 7, 26, 7, 854283, tzinfo=datetime.timezone.utc), content='**Event Overview:**\n- **Event Type:** Formal Dinner Party\n- **Number of Guests:** 30\n- **Dietary Restrictions:** None\n- **Menu Preferences:** A mix of textures and flavors\n\n**Menu Outline:**\n\n1. **Appetizer Course:**\n   - *Option 1:* Chilled Cucumber Avocado Soup with a touch of lime, served with a textural garnish of c

## External Team

In [4]:
## Society of mind team

outer_culinary_team = SocietyOfMindAgent(
    name = 'CulinaryTeamAgent',
    team = culinary_team,
    model_client=model_client,
    description = 'A culinary team agent which will build the menu and receipes.',
    response_prompt="""
    You are the CulinaryTeamAgent, acting as a single, specialized expert within the outer team. 
    Your sole purpose is to produce a finalized, human-approved menu from the inner 'Culinary Team' workflow. 
    You do not generate recipes yourself. 
    Once the inner team has successfully created and gotten a menu approved by the human, you will provide this final menu and ingredients required to the LogisticsAgent as a complete input.
    You will also save the Complete Menu, Ingredients and Recipes as a .docx file.
    If the human rejects the menu, you will manage the inner team's revision process until a menu is approved.
    """
)


In [70]:
## Google Serper API Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from pydantic import BaseModel, Field

import os

class SearchWebOutput(BaseModel):
    item : str = Field(...,description = "Name and quntity of the item")
    price: int = Field(..., desciption = "Price of the item")


def search_web(query: str):
    """Search the web for getting the latest product cost using Serper API
    """
    try:
        api_key = os.getenv("SERPER_API_KEY")
        search_tool_wrapper = GoogleSerperAPIWrapper(serper_api_key=api_key)
        
        return search_tool_wrapper.results(query=query, )
    except Exception as e:
        return f"Search failed : {str(e)}"
    
search_web_agent = AssistantAgent(
    name="SearchWeb",
    model_client=model_client,
    description="Agent to seach the web using search_web tool",
    system_message="You are an agent to assist with user query", #with access to search_web tool. You will search the web for each call and return the result in the defined output format",
    tools=[search_web],
    tool_call_summary_format=SearchWebOutput,
    #output_content_type=SearchWebOutput
    
)
    

In [71]:
result = await search_web_agent.run(task="Average 1 kg potato ingredients prices in Kelowna, Canada")
result.messages[-1].content

AttributeError: format

In [39]:
from autogen_ext.tools.langchain import LangChainToolAdapter


In [32]:
## Logistic Agent

logistic_agent = AssistantAgent(
    name = 'LogisticAgent',
    model_client=model_client,
    description='An agents which creates a complete blueprint.',
    system_message="""
    You are the LogisticsAgent, responsible for turning a finalized menu into a concrete action plan. 
    
    Your task is to receive the approved menu from the CulinaryTeamAgent and generate a detailed logistical blueprint. 
    
    This includes:

    1. Shopping List: A comprehensive list of all ingredients and quantities.

    2. Preparation Timeline: A step-by-step schedule from food prep to serving.

    3. Equipment List: A list of necessary cooking tools and equipment.
    
    Pass this complete logistics plan to the BudgetAgent.
    """,
    reflect_on_tool_use=True
)

## budget agent
budget_agent = AssistantAgent(
    name = 'BudgetAgent',
    model_client=model_client,
    tools=[search_web],
    description='An agent which calculates total budget of the event.',
    system_message="""
    You are the BudgetAgent, a financial analyst for the event. 
    Your job is to calculate the total cost of the event based on the logistics plan and approved menu. 
    You will take the shopping list and equipment needs and generate a comprehensive financial report. 
    
    This report must include:

    1. Search the web for estimated latest prices of ingredients using tool "search_web".

    2. Search the web for estimated cost of equipment rentals using tool "search web" (if any).
    
    3. Calculate the labour cost.
    
    4. List down the cost of each ingredients menu wise

    5. A final, total budget for the event.
    
    **Make sure to provide the output in a propert markdown format**.
    
    Provide this final budget report along with the logistics plan to the UserProxyAgent for final review and approval.
    """
)

final_approval = UserProxyAgent(
    name='FinalApproval',
    description='A proxy agent that represent the user for final approval',
    input_func=input
)

In [36]:
search_web("Vegetable samosas ingredients prices")

"Veggie Samosas. 2 reviews. Regular price $7.69 Sale price Unit price /per ... Stuffing Ingredients: Potatoes (sodium metabisulphite), Onions, Peas, Canola ... Samosa: Enriched Wheat Flour, Water, Potatoes, Carrots, Green Peas, Onions, Vegetable Oil, Spices And Herbs, Salt, White Vinegar, Mango Powder, Turmeric. Ingredients: Potatoes, white flour, canola oil, green peas, ginger, salt, cumin seeds, garam masala (clove, cinnamon, coriander, cumin seeds, cardamom black, ... Ingredients. Potatoes, Carrots, Peas, Lentils, Onion, Canola Oil, Cilantro, Salt, Spice, Garlic, Citric Acid. Pastry: Water, Enriched Wheat Flour, Salt Vinegar. Samosa: Enriched Wheat Flour, Water, Potatoes, Carrots, Green Peas, Onions, Vegetable Oil, Spices And Herbs, Salt, White Vinegar, Mango Powder, Turmeric. A corner to corner filled samosa with Carrots, Peas, Onions, Potato's, and an enticing blend of Indian spices that will kick your taste buds into overdrive. It hits all the right notes! 5 jumbo samosas Spice L

In [37]:
result = await budget_agent.run(task="Vegetable samosas ingredients prices")
result.messages[-1].content

"Mini Vegetable Samosas. $3.99/8.5 Oz. Frozen Appetizers & Sides Sukhi's Vegetable Samosa Street Wrap, Mild ingredients ... Current price: $1.89$189. Original Price: $3.79. $3.79. Amy's Kitchen ... Find WHOLE FOODS MARKET Potato Vegetable Samosas at Whole Foods Market. Get nutrition, ingredient, allergen, pricing and weekly sale information! A blend of potatoes, peas, onions, carrots, squash, peppers and spices in a triangle spring roll wrapper. (1.0 oz. each) A corner to corner filled samosa with Carrots, Peas, Onions, Potato's, and an enticing blend of Indian spices that will kick your taste buds into overdrive. Frozen Meals Deep Indian Kitchen Samosas, Potato & Pea ingredients. Frozen ... Current price: $7.09$709. Deep Indian Kitchen Naan, Garlic. 3 ct · Current ... Stuffing Ingredients: Potatoes (contains sodium metabisulfite), Onions, Peas, Canola oil, Spices, Salt, Herbs, Ginger. Pastry Ingredients: Enriched wheat flour, ... I think this was $3.59 for about 12 little samosas. Imo

In [33]:
await Console(budget_agent.run_stream(task ="Vegetable pakoda ingredients prices"))

---------- TextMessage (user) ----------
Vegetable pakoda ingredients prices
---------- ToolCallRequestEvent (BudgetAgent) ----------
[FunctionCall(id='call_rNQ9go7rZwjETeqVPvJiKkfH', arguments='{"query":"Vegetable pakoda ingredients prices"}', name='search_web')]
---------- ToolCallExecutionEvent (BudgetAgent) ----------
[FunctionExecutionResult(content='All natural, no artificial ingredients or preservatives. Buy Bulk Online: Vegetable Pakoras Bulk Bags. Regular price $37.70. A traditional Indian delicacy. Fresh julienne cauliflower, onions, potatoes, spinach, and jalapeno peppers, accented with an exotic curry blend. $5.99 Current price is: $5.99. Mixed vegetables dipped in a spicy chickpea batter and fried. Vegetable Pakoras quantity. Add to cart. Pakora are crispy fritters made with vegetables like onions, potatoes, gram flour, spices and herbs. Also known as pikora or pakura, it is a popular snack and ... 1 cup chickpea flour (also known as besan or gram flour) · 5 tablespoons wa

TaskResult(messages=[TextMessage(id='2bed6002-bf03-4426-ae86-5d4ec23051ba', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 13, 7, 9, 13, 864893, tzinfo=datetime.timezone.utc), content='Vegetable pakoda ingredients prices', type='TextMessage'), ToolCallRequestEvent(id='47240750-4743-4c3a-aade-cb13e7fead81', source='BudgetAgent', models_usage=RequestUsage(prompt_tokens=243, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 8, 13, 7, 9, 14, 543702, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_rNQ9go7rZwjETeqVPvJiKkfH', arguments='{"query":"Vegetable pakoda ingredients prices"}', name='search_web')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='d029a25a-7137-43a1-8b4c-98c4d2a8f279', source='BudgetAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 13, 7, 9, 15, 349060, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='All natural, no artificial ingredient

In [8]:
## Termination condition
outer_team_termination = TextMentionTermination('APPROVE')

## Defining outer team
event_manager = RoundRobinGroupChat(
    participants=[outer_culinary_team, logistic_agent, budget_agent, final_approval],
    termination_condition=outer_team_termination
)



In [10]:
from autogen_agentchat.ui import Console
task = """Please plan a menu for a formal dinner party for 5 people. The budget for food is $50 per person.Indian cuisine
"""
await Console(event_manager.run_stream(task = task))

---------- TextMessage (user) ----------
Please plan a menu for a formal dinner party for 5 people. The budget for food is $50 per person.Indian cuisine

---------- TextMessage (PlannerAgent) ----------
**Event Brief: Formal Dinner Party**

- **Number of Guests:** 5
- **Dietary Restrictions:** None specified
- **Event Type:** Formal
- **Food Budget:** $50 per person (Total: $250)

**Menu Outline:**

1. **Appetizer:**
   - **Choice:** Vegetable Samosas (3 per person)
  
2. **Main Course:**
   - **Choices:** 
     - Chicken Tikka Masala (3 pieces per person)
     - Paneer Butter Masala (for vegetarian option)
   - **Accompaniments:**
     - Garlic Naan (2 pieces per person)
     - Jeera Rice (1 cup per person)
  
3. **Side:**
   - **Choice:** Mixed Vegetable Raita (1 small bowl per person)
  
4. **Dessert:**
   - **Choice:** Gulab Jamun (2 pieces per person)

5. **Beverages:**
   - **Choices:**
     - Mango Lassi (1 glass per person)
     - Masala Chai (1 cup per person)

This structured

C:\Users\ashut\AppData\Local\Temp\ipykernel_27692\1598312230.py:8: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleSearchAPIWrapper``.
  search_tool_wrapper = GoogleSearchAPIWrapper()


[FunctionCall(id='call_7gmbI45riUVXunwktuaSBBHW', arguments='{"query": "vegetable samosas ingredients prices"}', name='search_web'), FunctionCall(id='call_QDVqZPRxUzDSpYDPfv9xajgr', arguments='{"query": "chicken tikka masala ingredients prices"}', name='search_web'), FunctionCall(id='call_0GrKcQAQ8xq6WNdJ1Z1jaspT', arguments='{"query": "paneer butter masala ingredients prices"}', name='search_web'), FunctionCall(id='call_TrUREW60ylI8CwlMWQ5GVVPb', arguments='{"query": "garlic naan ingredients prices"}', name='search_web'), FunctionCall(id='call_eDTaXJWr31IglAsZULgsYDQc', arguments='{"query": "jeera rice ingredients prices"}', name='search_web'), FunctionCall(id='call_YNgvh8I51rooHHydY4Xuxdmn', arguments='{"query": "mixed vegetable raita ingredients prices"}', name='search_web'), FunctionCall(id='call_XMRvnFGrAnGTqLj4YHVEgeVT', arguments='{"query": "gulab jamun ingredients prices"}', name='search_web'), FunctionCall(id='call_42Ui757yYzrX3mXcJdRanTG1', arguments='{"query": "mango lassi i

TaskResult(messages=[TextMessage(id='4b3b95bf-22cc-4af2-9a1e-1303e70ad75f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 13, 6, 11, 5, 362694, tzinfo=datetime.timezone.utc), content='Please plan a menu for a formal dinner party for 5 people. The budget for food is $50 per person.Indian cuisine\n', type='TextMessage'), TextMessage(id='9e2e81e2-1bba-41a0-808c-d07fcb6dfb76', source='PlannerAgent', models_usage=RequestUsage(prompt_tokens=164, completion_tokens=276), metadata={}, created_at=datetime.datetime(2025, 8, 13, 6, 11, 12, 14596, tzinfo=datetime.timezone.utc), content="**Event Brief: Formal Dinner Party**\n\n- **Number of Guests:** 5\n- **Dietary Restrictions:** None specified\n- **Event Type:** Formal\n- **Food Budget:** $50 per person (Total: $250)\n\n**Menu Outline:**\n\n1. **Appetizer:**\n   - **Choice:** Vegetable Samosas (3 per person)\n  \n2. **Main Course:**\n   - **Choices:** \n     - Chicken Tikka Masala (3 pieces per person)\n     